In [1]:
# need python 3.12+ for typing
!python -V

Python 3.12.6


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.sokoban_solver import SokobanEdge, SokobanNode, SokobanSolver
from boards.utilities import load_json_to_dict

**Solver to do**
- Get the solver working on the "11" problem
    - "Working" = "In well under 1 second"
    - Maybe start with a few other demo puzzles
    - Might want the ability to create heuristics? Like by subclassing to specify how the queue is implemented.
    - So replace the queue list with a generic queue object (priority queue?...) that different solver implement differently.
    - Ex. heuristic: past a depth of len(board.boxes)?
    - See import queue; queue.PriorityQueue
- Create unit tests (runs solver on some test problems)

**Generator to do**
- Learn the papers
    - https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=af6c8e280946f019c7f632c8281a35ddf1fdc9b0
    - https://ianparberry.com/techreports/LARC-2011-01.pdf
    - Also do a bit more literature search
- Start implementing techniques

## Basic demos

In [4]:
filename = './boards/board_trivial.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node1.player_component)
print('Avaiable pushes: ', node1.edges)
print('Is solved?: ', node1.solved)

print(node1)

NameError: name 'node1' is not defined

In [ ]:
filename = './boards/board_simple1.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node1 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node1.player_component)
print('Avaiable pushes: ', node1.edges)
print('Is solved?: ', node1.solved)

print(node1)

In [ ]:
filename = './boards/board_simple2.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node2 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node2.player_component)
print('Avaiable pushes: ', node2.edges)
print('Is solved?: ', node2.solved)

print(node2)

In [ ]:
filename = './boards/board_simple3.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node3 = SokobanNode(airs, boxes, goals, player)
print('Player component: ', node3.player_component)
print('Avaiable pushes: ', node3.edges)
print('Is solved?: ', node3.solved)

print(node3)

In [ ]:
node1 == node2, node1 == node3, node2 == node3

In [ ]:
solver = SokobanSolver(node)
solver.solve()
print(solver.solution)

In [ ]:
solver = SokobanSolver(node1)
solver.solve()
print(solver.solution)

In [ ]:
solver = SokobanSolver(node2)
solver.solve()
print(solver.solution)

In [ ]:
solver = SokobanSolver(node3)
solver.solve()
print(solver.solution)

## Real puzzles

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=1

In [ ]:
filename = './boards/board_v=1&seed=1565730199&level=1.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

In [ ]:
solver = SokobanSolver(node)
solver.solve()
print(solver.solution)

https://linusakesson.net/games/autosokoban/?v=1&seed=1565730199&level=11

In [5]:
filename = './boards/board_v=1&seed=1565730199&level=11.json'
d = load_json_to_dict(filename)

airs = d['airs']
boxes = d['boxes']
goals = d['goals']
player = d['player']

node = SokobanNode(airs, boxes, goals, player)
print(node)

█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █   █ █ █ █ █ █
█ █ █   ·         █ █ █
█ █ █   █   □ · □   █ █
█ █ · □ □   ■ ·     █ █
█ █     □   █ █ █ █ █ █
█ █   █ █   █ █ █ █ █ █
█ █ · ○     █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █
█ █ █ █ █ █ █ █ █ █ █ █


In [6]:
# unable to find the solution in reasonable time
solver = SokobanSolver(node)
solver.solve(max_seconds=1)

301

In [7]:
solver.n_iters_overall

4926

In [8]:
solver.n_seconds_overall

1.0002968311309814

## Bottom

In [ ]:
solver.step()
print(len(solver.queue))
print()
print(solver.queue.get_str_end_nodes())